In [36]:
# file_path = './data/spam_data/train/ham/1387.2000-06-17.farmer.ham.txt'

# with open(file_path, "r") as infile:
#     ham_sample = infile.read()

# print(ham_sample)

In [37]:
# file_path = './data/spam_data/train/spam/1778.2004-08-09.GP.spam.txt'

# with open(file_path, "r") as infile:
#     spam_sample = infile.read()

# print(spam_sample)

In [38]:
# # Buh text file iig unshaad ham/spam gedgeer ni label uusgene
# # spam bol 1
# # ham bol 0
# import os
# import glob

# mails = []
# labels = []

# # spam e-mail uudiig unshina
# file_path = './data/spam_data/train/spam/'
# for filename in glob.glob(os.path.join(file_path, '*.txt')):
#     with open(filename, "r", encoding="ISO-8859-1") as infile:
#         mails.append(infile.read())
#         labels.append(1) 

# print("--------------SPAM------------- \n", mails[0], labels[0], "\n--------------------")


In [1]:
# file_path = "./data/spam_data/train/ham/"
# for filename in glob.glob(os.path.join(file_path, "*.txt")):
#     with open(filename, "r", encoding="ISO-8859-1") as infile:
#         mails.append(infile.read())
#         labels.append(0)

# print("--------------------HAM------------------ \n", mails[1], labels[1], "\n-----------------------------")

In [2]:
from typing import NamedTuple
from math import *
from pathlib import Path
import re
import glob
from random import shuffle
from collections import *
from typing import *


spam_data_path = './data/train_data/train/spam/'
ham_data_path = './data/train_data/train/ham/'

spam_test_path = "./data/train_data/dev/spam"
ham_test_path = "./data/train_data/dev/ham/"

class Message(NamedTuple):
    text: str
    is_spam: bool

In [3]:
spam_message_paths = glob.glob(str(spam_data_path + '/*.txt'))
ham_message_paths  = glob.glob(str(ham_data_path + '/*.txt'))

print("spam", len(spam_message_paths))
print("ham", len(ham_message_paths))
message_paths = ham_message_paths + spam_message_paths
len(message_paths)

spam 1000
ham 1000


2000

In [4]:
spam_message_paths_test = glob.glob(str(spam_test_path + '/*.txt'))
ham_message_paths_test  = glob.glob(str(ham_test_path + '/*.txt'))

print("spam", len(spam_test_path))
print("ham", len(ham_test_path))
test_message_pathz = spam_message_paths_test + ham_message_paths_test
len(test_message_pathz)


spam 26
ham 26


1000

In [5]:
messages = []

for path in message_paths:
    
    with open(path, errors='ignore') as file:
        # is_spam = True if "spam" in path else False
        if "spam" in path:
            is_spam = True
        else: 
            is_spam = False
    
        text: str = file.readline().replace('Subject:', '').strip()
        messages.append(Message(text, is_spam))

In [6]:
test_messages = []

message_paths = test_message_pathz
for path in message_paths:
    
    with open(path, errors='ignore') as file:
        # is_spam = True if "spam" in path else False
        if "spam" in path:
            is_spam = True
        else: 
            is_spam = False
          
        # print(path, is_spam)
        text: str = file.readline().replace('Subject:', '').strip()
        test_messages.append(Message(text, is_spam))

In [7]:
len(test_messages)

1000

In [8]:
def tokenize(text: str):
    words = []
    # tokenizer hiihdee regex eer haigaad 
    # olson bug ugeeree guigeed bugdeng ni lowercase bolgood
    # words list ruu nemne
    for word in re.findall(r'[A-Za-z0-9\']+', text):
        if len(word) >= 2:
            words.append(word.lower())
    return set(words)

In [9]:
# scikit learnii 80, 20 oor ni huvaadag code iig garaar bichiv
# assert hiij testlev.

# def train_test_split(messages: List[Message], pct=0.8) -> Tuple[List[Message], List[Message]]:
#     shuffle(messages)
#     num_train = int(round(len(messages) * pct, 0))
#     return messages[:num_train], messages[num_train:]

In [25]:
class NaiveBayes:
    def __init__(self, k=1):
        self.k = k
        self.num_spam_messages: int = 0
        self.num_ham_messages: int = 0
        self.num_word_in_spam = defaultdict(int)
        self.num_word_in_ham = defaultdict(int)
        self.spam_words = set()
        self.ham_words = set()
        self.words = set()

    # train hesgiin codenii hiij bga ni 
    # label hiigdsen message uudiig avaad deegur ni iterate hiij message bolgonoor guine
    # teriige tokenize hiij ug ug bolgono.
    # ahiad itrate hiij guigeed constructor deer bga state uudiigee update hiine
    def train(self, messages):
        for msg in messages:
            tokens = tokenize(msg.text)
            self.words.update(tokens)
            if msg.is_spam:
                self.num_spam_messages += 1
                self.spam_words.update(tokens)
                for token in tokens:
                    self.num_word_in_spam[token] += 1
            else:
                self.num_ham_messages += 1
                self.ham_words.update(tokens)
                for token in tokens:
                    self.num_word_in_ham[token] += 1                
    
    # ugsun ug ni spam baih magadlaliig end tootsoj bna
    
    # hervee ugsun word ug ni spam dotor baihgui bol (P | S) = 0 buyu 0*0 bolchino
    # eniig boliulahiin tuld k factor hereglene
    # P (W | S) = k + (W | Spam) / (2 * k) + total_spam

    # k iin tuslamjtai magadlal ni 0 bolohgui. 0 tei oiroltsoo too garna.

    
    def p_word_spam(self, word):
        return (self.k + self.num_word_in_spam[word]) / ((2 * self.k) + self.num_spam_messages)
    
    def p_word_ham(self, word):
        return (self.k + self.num_word_in_ham[word]) / ((2 * self.k) + self.num_ham_messages)
    
    # p
    def predict(self, text):
        text_words = tokenize(text)
        log_p_spam = 0.0
        log_p_ham = 0.0

        for word in self.words:
            p_spam = self.p_word_spam(word)
            p_ham = self.p_word_ham(word)
            if word in text_words:
                log_p_spam += log(p_spam)
                log_p_ham += log(p_ham)
            else:
                log_p_spam += log(1 - p_spam)
                log_p_ham += log(1 - p_ham)

        p_if_spam = exp(log_p_spam)
        p_if_ham = exp(log_p_ham)
        return p_if_spam / (p_if_spam + p_if_ham)
    
    def predict_ham(self, text):
        text_words = tokenize(text)
        log_p_spam = 0.0
        log_p_ham = 0.0

        for word in self.words:
            p_spam = self.p_word_spam(word)
            p_ham = self.p_word_ham(word)
            if word in text_words:
                log_p_spam += log(p_spam)
                log_p_ham += log(p_ham)
            else:
                log_p_spam += log(1 - p_spam)
                log_p_ham += log(1 - p_ham)

        p_if_spam = exp(log_p_spam)
        p_if_ham = exp(log_p_ham)
        return p_if_ham / (p_if_spam + p_if_ham)

SyntaxError: invalid syntax (431731492.py, line 67)

In [24]:

# Splitting our Enron messages into a `train` and `test` set
# train, test = train_test_split(messages)
train = messages
test = test_messages


In [21]:

model = NaiveBayes()
model.train(train)

print(f'Train data-nd baigaa Spam message uudiin too: {model.num_spam_messages}')
print(f'Train data-nd baigaa Ham message uudiin too: {model.num_ham_messages}')
print(f'Hamgiin ih spam gej toologdson ugnuud \n: {Counter(model.num_word_in_spam).most_common(50)}')

Train data-nd baigaa Spam message uudiin too: 1000
Train data-nd baigaa Ham message uudiin too: 1000
Hamgiin ih spam gej toologdson ugnuud 
: [('you', 100), ('the', 92), ('your', 88), ('to', 86), ('for', 76), ('re', 60), ('and', 46), ('on', 46), ('in', 43), ('get', 41), ('is', 38), ('of', 36), ('this', 36), ('with', 34), ('new', 32), ('at', 32), ('online', 29), ('we', 28), ('it', 27), ('no', 27), ('software', 27), ('can', 26), ('from', 24), ('meds', 22), ('free', 22), ('want', 21), ('all', 21), ('best', 21), ('cialis', 20), ('cheap', 20), ('now', 19), ('soft', 19), ('00', 19), ('stock', 18), ('time', 17), ('here', 17), ('low', 17), ('viagra', 17), ('pharmacy', 17), ('are', 17), ('pain', 17), ('more', 16), ('off', 16), ('me', 16), ('prices', 16), ('need', 15), ('th', 15), ('hot', 14), ('office', 14), ('xp', 14)]


In [22]:
# Test datagaas spam uudiig ni yalgay
spam_messages = [item for item in test if item.is_spam]
spam_messages[:5]

[Message(text='stop the aging clock', is_spam=True),
 Message(text='discover you made money while you were sleeping aaer xchxa', is_spam=True),
 Message(text='onlinepharmacycheap', is_spam=True),
 Message(text='cohere handicraftsman', is_spam=True),
 Message(text='! ebay reguiar verification', is_spam=True)]

In [15]:
message = spam_messages[12].text
print(f'"{message}" spam baih magadlal.')
model.predict(message)

"get your viagr $ a today !" spam baih magadlal.


0.9989053416549496

In [16]:
# test set ees ham oguulberuudiig avch bna
ham_messages = [item for item in test if item.is_spam != True] 
ham_messages[:5]

[Message(text='re : resume attached', is_spam=False),
 Message(text='e - mail', is_spam=False),
 Message(text='get a $ 25 certificate just for responding to this e - mail', is_spam=False),
 Message(text='texas nom , lp dba garrison , ltd', is_spam=False),
 Message(text='june vacation', is_spam=False)]

In [17]:
message = ham_messages[11].text

print(f'"{text}" spam baih magadlal.')
model.predict(message)

"sarco lateral and crow o ' connor meters" spam baih magadlal.


0.007030845822791404

In [18]:
message = ham_messages[11].text
model.predict_ham(message)

AttributeError: 'NaiveBayes' object has no attribute 'predict_ham'